In [ ]:
%matplot inline

## 모델 매개변수 최적화

데이터에 매개변수를 최적화하여 모델을 학습하고, 테스트 하는 과정

모델 학습은 반복적인 과정을 거침
* 각 반복 단계에서 모델은 출력을 추측
* 추측과 정답사이의 loss를 계산
* 매개변수에 대한 오류의 도함수를 수집
* 경사하강법으로 파라미터를 **최적화**

## 기본 코드

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NN()


## 하이퍼파라미터

모델 최적화 과정을 제어할 수 있는 조절 가능한 매개변수
서로 다른 하이퍼파라미터 값은 모델학습과 수렴율에 영향을 미칠 수 있다.

학습시 다음의 하이퍼 파라미터 정의
* 에폭 수 (epoch) - 데이터셋을 반복하는 횟수
* 배치 크기 (batch size) - 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플 수
* 학습률 (learning rate) - 각 배치/에폭에서 모델의 매개변수를 조절하는 비율. 값이 작을수록 학습속도가 느려지고, 값이 클 수록 학습 중 에측할 수 없는 동작이 발생 가능함

In [ ]:
learinig_rate = 1e-3
batch_size = 64
epochs = 5

## 최적화 단계

하이퍼 파라미터 설정 이후 최적화  단계를 통해 모델을 학습 후 최적화

각 반복단계 = 에폭

하나의 에폭은 두 부분으로 구성된다

* 학습단계 -> 학습용 데이터셋을 반복하고 최적의 매개변수로 수렴
* 검증/테스트 단계 -> 모델 성능이 개선되고 있는지 보기 위해 데이터셋을 반복

아래 설명하는 개념들은 학습단계에 포함

## 손실함수

일반적인 손실함수는 MSE, nll, Crossentropy loss 등이 있음

In [ ]:
# 손실함수 초기화
loss_fn = nn.CrossEntropyLoss()

## 옵티마이저

최적화는 각 학습 단계에서 오류 줄이기 위해 파라미터 조절하는 과정.

최적화 알고리즘을 (여기서는 확률적 경사하강범 SGD) 정의함
모든 최적화 절차는 optimizer 객체에 캡슐화(encapsulate 됨)



In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=learinig_rate)

학습 단계에서 최적화는 세 단계
* optimizer.zero.grad 를 호출해 매개변수 변화도 재설정, 기본적으로 변화도는 더해지기 떄문에 중복계산을 막기 위해 0으로 설정
* loss.backward() 호출해 예측 손실을 역전파 함. 각 매개변ㅅ수에 대한 손실의 변화도를 저장
* 변화도 계산 이후에 optimizer.step 호출해 역전파 단계에서 수집된 변화도로 매개변수 조정

## 전체 구현

최적화 코드 반복해 수행하는 train loop, 데스트 데이터로 모델의 성능 측정하는 test loop 정의

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측과 손실 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item() # 손실 값 누적
            corect += (pred.argmax(1) == y).type(torch.float).sum().item() # 정확히 예측된 값 누적

    test_loss /= num_batches # 테스트 손실값 평균 계산
    correct /= size # 정확도 평균계산
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

손실함수, 옵티마이저 초기화 이후 train loop, test loop 에 전달함. 모델의 성능 향상을 알아보기 위해 자유롭게 에폭 수를 증가시킬 수 있음

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learinig_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
print("done!!")